In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats


pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_columns', None)
plt.style.use('fivethirtyeight')
plt.rcParams['figure.facecolor'] = 'white'

## Task 1:

The external data needs coordinate attributes

In [2]:
df = pd.read_csv('data/airline_delay.csv') #<-- Original dataset

# DATA SOURCE 1
df_airports = pd.read_csv('data/us-airports.csv')
df_airports = df_airports.drop(df_airports.index[0]).reset_index(drop=True)

# DATA SOURCE 2
df_airports2 = pd.read_csv('data/airports.csv')
df_airports2.rename(columns={'LATITUDE':'latitude_deg', 'LONGITUDE':'longitude_deg', 'AIRPORT':'name', 'STATE':'local_region', 'CITY':'municipality'}, inplace=True)
df_airports2.drop(columns=['COUNTRY'], inplace=True)

# Data gathered from https://ourairports.com/countries/US/ & https://data.humdata.org/dataset/ourairports-usa?

In [3]:
display(df.sample(2))
print(f"Number of unique airports in original dataset: {df['airport'].nunique()}")

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
1087,2020,12,OO,SkyWest Airlines Inc.,LAN,"Lansing, MI: Capital Region International",40.0,9.0,4.00,5.0,0.00,0.0,0.0,0.0,0.0,1122.0,214.0,908.0,0.0,0.0,0.0
862,2020,12,OH,PSA Airlines Inc.,CAK,"Akron, OH: Akron-Canton Regional",97.0,6.0,3.04,0.0,2.96,0.0,0.0,0.0,0.0,174.0,108.0,0.0,66.0,0.0,0.0


Number of unique airports in original dataset: 360


In [4]:
display(df_airports[df_airports['local_code'].str.len() == 3].sample(2))
print(f"Number of unique airports in first dataset: {df_airports[df_airports['local_code'].str.len() == 3]['local_code'].nunique()}")

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,region_name,iso_region,local_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated
3902,26354,Z48,small_airport,Bear Creek 3 Airport,63.5733160384,-156.149454117,740,NaN,United States,US,Alaska,US-AK,AK,Bear Creek,0,Z48,BCC,Z48,NaN,https://en.wikipedia.org/wiki/Bear_Creek_3_Air...,NaN,50,2012-08-11T18:09:15+00:00
2766,21277,KU82,small_airport,Council Municipal Airport,44.750286,-116.445145,2963,NaN,United States,US,Idaho,US-ID,ID,Council,0,NaN,NaN,U82,NaN,NaN,NaN,50,2021-11-26T11:20:20+00:00


Number of unique airports in first dataset: 5441


In [5]:
display(df_airports2.sample(2)) 
print(f"Number of unique airports in second dataset: {df_airports2['IATA'].nunique()}")

,IATA,name,municipality,local_region,latitude_deg,longitude_deg
157,ESC,Delta County,Escanaba,MI,45.72267,-87.093731
228,LFT,Lafayette Regional,Lafayette,LA,30.20528,-91.987655


Number of unique airports in second dataset: 341


In [12]:
missing_in_1 = ~df["airport"].isin(df_airports["local_code"])
missing_in_2 = ~df["airport"].isin(df_airports2["IATA"])
missing_values = df[missing_in_1 & missing_in_1]

print("Values from df['airport'] missing in both df_airports['local_code'] and df_airports2['IATA']:")
missing_values[["airport", "airport_name"]].drop_duplicates().reset_index(drop=True)

Values from df['airport'] missing in both df_airports['local_code'] and df_airports2['IATA']:


,airport,airport_name
0,SJU,"San Juan, PR: Luis Munoz Marin International"
1,STT,"Charlotte Amalie, VI: Cyril E King"
2,STX,"Christiansted, VI: Henry E. Rohlsen"
3,FCA,"Kalispell, MT: Glacier Park International"
4,AZA,"Phoenix, AZ: Phoenix - Mesa Gateway"
5,SCE,"State College, PA: University Park"
6,USA,"Concord, NC: Concord Padgett Regional"
7,MQT,"Marquette, MI: Sawyer International"
8,YUM,"Yuma, AZ: Yuma MCAS/Yuma International"
9,GUM,"Guam, TT: Guam International"


In [6]:
df_full = pd.merge(df, df_airports[['local_code', 'latitude_deg', 'longitude_deg', 'name', 'local_region', 'municipality']], left_on='airport', right_on='local_code', how='left')
df_full = pd.merge(df_full, df_airports2, left_on='airport', right_on='IATA', how='left', suffixes=('', '_2'))

for column in df_airports2.columns:
    if column == "IATA":
        continue
    df_full[column] = df_full[column].fillna(df_full[column+"_2"])
df_full.drop([x+"_2" for x in df_airports2.columns if x != "IATA"], axis=1, inplace=True)

df_full.sample(3)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,local_code,latitude_deg,longitude_deg,name,local_region,municipality,IATA
3023,2019,12,AA,American Airlines Inc.,ORF,"Norfolk, VA: Norfolk International",147.0,20.0,9.02,0.00,3.73,0.00,7.25,0.0,1.0,872.0,359.0,0.0,120.0,0.0,393.0,ORF,36.895341,-76.201,Norfolk International Airport,VA,Norfolk,ORF
2181,2019,12,YV,Mesa Airlines Inc.,OAK,"Oakland, CA: Metropolitan Oakland International",23.0,9.0,1.26,0.00,0.28,0.10,7.36,2.0,0.0,838.0,98.0,0.0,10.0,8.0,722.0,OAK,37.720085,-122.221184,Metropolitan Oakland International Airport,CA,Oakland,OAK
3115,2019,12,AS,Alaska Airlines Inc.,SAN,"San Diego, CA: San Diego International",747.0,193.0,41.48,0.32,80.63,0.59,69.98,9.0,3.0,9449.0,2206.0,15.0,2940.0,23.0,4265.0,SAN,32.7336006165,-117.190002441,San Diego International Airport,CA,San Diego,SAN


## Question 1

What are the most delayed airports?

In [82]:
airport_delays = df.groupby(['airport', 'airport_name']).agg({'arr_flights': 'sum', 'arr_del15': 'sum'}).reset_index()

airport_delays['delay_percentage'] = (airport_delays['arr_del15'] / airport_delays['arr_flights']) * 100
airport_delays = airport_delays.sort_values(by='delay_percentage', ascending=False)

top_airports = airport_delays.head(20)

fig = px.bar(top_airports, x='delay_percentage', y='airport', text='delay_percentage',
             hover_data=['airport_name'], orientation='h',
             title='Top Airports Delayed')

fig.update_layout(
    xaxis_title="Percentage of Delays (%)",
    yaxis_title="Airport Code",
    xaxis=dict(tickformat=".2f"),
    height=1200,
)

fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside', marker_color='#CF0A2C',)
fig.show()

## Task: Aggregate the Data by Airport

In [77]:
airport_delays_agg = df_full.groupby('airport').agg({
    'arr_flights': 'sum',
    'arr_del15': 'sum',
    'arr_delay': 'sum'
}).reset_index()

airport_delays_agg.rename(columns={'arr_flights': 'arr_flights_sum', 'arr_del15': 'arr_del15_sum', 'arr_delay': 'arr_delay_sum'}, inplace=True)
airport_delays_agg['delay_percentage'] = (airport_delays_agg['arr_del15_sum'] / airport_delays_agg['arr_flights_sum']) * 100
airport_delays_agg['delay_time_average'] = (airport_delays_agg['arr_delay_sum'] / airport_delays_agg['arr_del15_sum'])

# Merge with df_airports for coordinates and additional info
airport_delays = pd.merge(df_full, airport_delays_agg, on='airport', how='left')
airport_delays.dropna(subset=['delay_time_average', 'delay_percentage'], inplace=True)

# remove the outliers
z_scores_percentage = stats.zscore(airport_delays['delay_percentage'])
airport_delays = airport_delays[(z_scores_percentage < 3.5)]
z_scores_time = stats.zscore(airport_delays['delay_time_average'])
airport_delays = airport_delays[(z_scores_time < 3.5)]

# remove the NaN values
airport_delays.dropna(inplace=True)

airport_delays #.describe().T
airport_delays.describe().T

,count,mean,std,min,25%,50%,75%,max
year,3165.0,2019.457188,0.498242,2019.000000,2019.000000,2019.000000,2020.000000,2020.000000
month,3165.0,12.000000,0.000000,12.000000,12.000000,12.000000,12.000000,12.000000
arr_flights,3165.0,309.502370,874.048390,1.000000,35.000000,85.000000,201.000000,19713.000000
arr_del15,3165.0,52.804423,150.060072,0.000000,5.000000,13.000000,34.000000,2289.000000
carrier_ct,3165.0,16.565592,42.718510,0.000000,1.490000,4.820000,12.990000,697.000000
weather_ct,3165.0,1.478654,4.935943,0.000000,0.000000,0.070000,1.070000,89.420000
nas_ct,3165.0,16.830847,57.867021,0.000000,0.920000,3.000000,9.210000,1039.540000
security_ct,3165.0,0.140148,0.658925,0.000000,0.000000,0.000000,0.000000,17.310000
late_aircraft_ct,3165.0,17.789216,56.808259,0.000000,0.940000,3.420000,10.590000,819.660000
arr_cancelled,3165.0,2.974092,10.382749,0.000000,0.000000,0.000000,2.000000,224.000000


In [40]:
airport_delays.sample(2)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,local_code,latitude_deg,longitude_deg,name,local_region,municipality,IATA,arr_flights_sum,arr_del15_sum,arr_delay_sum,delay_percentage,delay_time_average,abs_difference
2115,2019,12,YV,Mesa Airlines Inc.,BTR,"Baton Rouge, LA: Baton Rouge Metropolitan/Ryan...",54.0,5.0,3.00,2.0,0.00,0.0,0.00,1.0,0.0,434.0,59.0,375.0,0.0,0.0,0.0,BTR,30.533199,-91.149597,Baton Rouge Metropolitan Airport,LA,Baton Rouge,BTR,977.0,126.0,7922.0,12.896622,62.873016,49.976394
2878,2019,12,9E,Endeavor Air Inc.,EWN,"New Bern/Morehead/Beaufort, NC: Coastal Caroli...",58.0,5.0,3.81,0.0,0.82,0.0,0.37,0.0,1.0,160.0,135.0,0.0,15.0,0.0,10.0,EWN,35.0730018616,-77.04290008539999,Coastal Carolina Regional Airport,NC,New Bern,EWN,302.0,45.0,2561.0,14.900662,56.911111,42.010449


In [79]:
initial_airport_delays = airport_delays.copy()

z_scores_percentage = stats.zscore(airport_delays['delay_percentage'])
filtered_airport_delays = airport_delays[(z_scores_percentage < 3.5)]
z_scores_time = stats.zscore(filtered_airport_delays['delay_time_average'])
filtered_airport_delays = filtered_airport_delays[(z_scores_time < 3.5)]

filtered_airport_delays.dropna(inplace=True)

removed_outliers = initial_airport_delays[~initial_airport_delays.index.isin(filtered_airport_delays.index)]

removed_outliers_display = removed_outliers[['name', 'arr_flights_sum', 'delay_percentage', 'delay_time_average']]
print("Removed Outliers:")
display(removed_outliers_display)

Removed Outliers:


,name,arr_flights_sum,delay_percentage,delay_time_average
15,Brunswick Golden Isles Airport,179.0,8.938547,112.437500
31,Central Wisconsin Airport,313.0,14.696486,119.586957
575,Central Nebraska Regional Airport,175.0,13.142857,118.347826
639,Abraham Lincoln Capital Airport,191.0,19.371728,127.783784
704,Central Wisconsin Airport,313.0,14.696486,119.586957
724,Central Nebraska Regional Airport,175.0,13.142857,118.347826
741,Lawton Fort Sill Regional Airport,181.0,16.574586,116.166667
951,Aspen-Pitkin County Airport (Sardy Field),1307.0,28.156083,101.565217
1026,Cheyenne Regional Jerry Olson Field,62.0,16.129032,111.400000
1038,Devils Lake Regional Airport,106.0,12.264151,134.307692


## Map Blueprint

In [30]:
def make_scatter_map(size, color, title):

    fig = go.Figure()

    fig.add_trace(go.Scattergeo(
        lon = airport_delays['longitude_deg'],
        lat = airport_delays['latitude_deg'],
        text = airport_delays['name'] + '<br>Delay Average: ' + airport_delays['delay_time_average'].round(1).astype(str) + 
            '<br>Delay Percentage: ' + airport_delays['delay_percentage'].round(1).astype(str) +
            '<br>Number of Flights: ' + airport_delays[size].astype(int).astype(str),

        marker = dict(
            size = airport_delays[size], 
            color = airport_delays[color],
            colorscale = 'RdYlGn_r',
            colorbar_title = '',
            line_color='grey',
            sizemode = 'area',
            sizeref = airport_delays[size].max() / 350, 
            sizemin = 3.5  # minimum marker size
        ),
        hoverinfo='text'
    ))

    fig.update_geos(
        resolution=110,
        showcountries=True, 
        landcolor="beige",
        countrycolor="black",
        subunitcolor="grey"
    )

    fig.update_layout(
        title=dict(text=title, x=0.5),
        geo=dict(
            scope='usa',
            projection_type='albers usa',
            lakecolor='#E5E4E2',
            bgcolor='rgba(255,255,255,0.8)'
        ),
        margin=dict(l=0, r=0, t=40, b=10),
        width=1000,
        height=500
    )
    fig.show()


## Map 1: By Time Delay

In [31]:
make_scatter_map(size='arr_flights_sum', color='delay_time_average', title="Average Time Delay by Airport")

## Map 2: By percentage of delays

In [48]:
make_scatter_map(size='arr_flights_sum', color='delay_percentage', title="Flight Delays by Airport")

## Task: Find Absolute Difference

Between average delay time and percentage of delay

In [68]:
aggregated_airport_delays = airport_delays.groupby('name').agg({
    'delay_percentage': 'mean',
    'delay_time_average': 'mean',
    'arr_flights': 'sum'
}).reset_index()

# top 5 airports highest delays
filtered_airport_delays = aggregated_airport_delays[aggregated_airport_delays['arr_flights'] > 15000].copy()
top_5_highest_delay_percentage = filtered_airport_delays.nlargest(4, 'delay_percentage')

# top 5 airports lowest delay_time_average and lowest delay_percentage
filtered_airport_delays = aggregated_airport_delays[aggregated_airport_delays['arr_flights'] > 30000].copy()
filtered_airport_delays['abs_difference'] = (filtered_airport_delays['delay_percentage'] - filtered_airport_delays['delay_time_average']).abs()
top_5_lowest_delay_time_and_percentage = filtered_airport_delays.nsmallest(4, 'abs_difference')


print("Top 5 Airports with the Highest Delay Percentage and More Than 15,000 Flights:")
display(top_5_highest_delay_percentage)
print("\nTop 5 Airports with the Lowest Delay Time Average and Lowest Delay Percentage with More Than 30,000 Flights:")
display(top_5_lowest_delay_time_and_percentage)

Top 5 Airports with the Highest Delay Percentage and More Than 20,000 Flights:


,name,delay_percentage,delay_time_average,arr_flights
193,Newark Liberty International Airport,27.695976,86.675204,16775.0
236,San Francisco International Airport,25.556105,80.701934,20230.0
160,Logan International Airport,25.242606,85.408696,17312.0
149,La Guardia Airport,23.707289,75.467451,17560.0



Top 5 Airports with the Lowest Delay Time Average and Lowest Delay Percentage with More Than 20,000 Flights:


,name,delay_percentage,delay_time_average,arr_flights,abs_difference
70,Denver International Airport,16.628275,64.426244,39505.0,47.797970
118,Hartsfield Jackson Atlanta International Airport,11.822216,60.753433,55438.0,48.931217
63,Dallas Fort Worth International Airport,18.667138,69.016098,45256.0,50.348960
45,Charlotte Douglas International Airport,13.726138,66.895036,33316.0,53.168898
